In [1]:
import os
import numpy as np
import pandas as pd

print(os.listdir("./data"))

df_train = pd.read_csv('./data/train_data.csv')
df_test =  pd.read_csv('./data/test_features.csv')

for u in df_train.columns:
    if df_train[u].dtype==bool:
        df_train[u]=df_train[u].astype('int')

df_train.head()

['train_data.csv', 'enron61702insiderpay.pdf', 'test_features.csv', '.~lock.train_data.csv#', 'sample_submission.csv']


,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,1,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,1,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,1,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,1,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,1,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [2]:
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(20))
na_check(df_train)

,Missing Ratio
loan_advances,98.230088
restricted_stock_deferred,91.150442
director_fees,88.495575
deferral_payments,75.221239
deferred_income,69.911504
long_term_incentive,56.637168
bonus,46.017699
from_messages,42.477876
from_poi_to_this_person,42.477876
from_this_person_to_poi,42.477876


In [3]:
na_check(df_test)

,Missing Ratio
loan_advances,93.939394
director_fees,87.878788
restricted_stock_deferred,75.757576
deferral_payments,66.666667
deferred_income,54.545455
long_term_incentive,48.484848
from_this_person_to_poi,36.363636
from_poi_to_this_person,36.363636
exercised_stock_options,36.363636
from_messages,36.363636


In [4]:
def logdf(cols):
    df_train[cols] = df_train[cols].map(lambda i: np.log(i) if i > 0 else 0)

def dffillmed(cols):
    df_train[cols] = df_train[cols].fillna(df_train[cols].median())


df_train = df_train.drop(['email_address', 'name'] , axis=1)
df_train['deferred_income'] = df_train['deferred_income'].map(lambda i: np.log(-i) if -i > 0 else 0)

logdf('bonus')
logdf('deferral_payments')
logdf('director_fees')
logdf('exercised_stock_options')
logdf('expenses')
logdf('long_term_incentive')
logdf('other')
logdf('restricted_stock')
logdf('salary')
logdf('total_payments')
logdf('total_stock_value')

logdf('loan_advances')
logdf('restricted_stock_deferred')

dffillmed('from_poi_to_this_person')
dffillmed('from_messages')
dffillmed('to_messages')
dffillmed('shared_receipt_with_poi')
dffillmed('from_this_person_to_poi')

df_train.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,14.375126,0.0,15.069526,0.0,16.800898,10.756838,18.0,42.0,4.0,0.0,14.296090,12.071621,1,14.826516,0.0,12.949523,864.0,905.0,13.132413,16.930915
1,15.538277,0.0,0.000000,0.0,16.773022,10.286571,108.0,88.0,30.0,0.0,14.467836,10.004328,1,15.738835,0.0,13.921003,2042.0,3627.0,15.976845,17.077203
2,12.206073,0.0,8.334952,0.0,14.300647,10.038193,39.0,13.0,14.0,0.0,0.000000,14.268701,1,13.675352,0.0,12.263605,91.0,225.0,14.510598,14.729244
3,13.592367,0.0,0.000000,0.0,0.000000,11.679575,45.0,28.0,7.0,0.0,13.309127,13.718451,1,13.800429,0.0,12.320762,599.0,1088.0,14.791055,13.800429
4,14.038654,0.0,12.478006,0.0,0.000000,10.486206,144.0,199.0,25.0,0.0,12.835492,6.186209,1,11.744251,0.0,12.389181,2188.0,2598.0,14.309778,11.744251


In [5]:
na_check(df_train)

,Missing Ratio


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

X = df_train.drop(['poi'] , axis=1)
y = df_train['poi'][:,np.newaxis]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)



lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=19,
                                  max_depth=6, learning_rate=0.03)
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

In [7]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((84, 19), (29, 19), (84, 1), (29, 1))

In [8]:
gdbt.fit(x_train, y_train.ravel())
gdbt_pred = gdbt.predict_proba(x_test)
gdbt_pred

array([[9.99925226e-01, 7.47736958e-05],
       [9.99923984e-01, 7.60155455e-05],
       [9.99940961e-01, 5.90386503e-05],
       [9.99913766e-01, 8.62337204e-05],
       [9.98618157e-01, 1.38184288e-03],
       [9.99827933e-01, 1.72066967e-04],
       [3.35677392e-02, 9.66432261e-01],
       [9.99937211e-01, 6.27889618e-05],
       [9.99940961e-01, 5.90393955e-05],
       [9.99929358e-01, 7.06418892e-05],
       [9.99940987e-01, 5.90131180e-05],
       [9.99933710e-01, 6.62899815e-05],
       [9.99932762e-01, 6.72375675e-05],
       [9.94929566e-01, 5.07043444e-03],
       [9.99772444e-01, 2.27556146e-04],
       [9.99924512e-01, 7.54878255e-05],
       [9.33955151e-01, 6.60448491e-02],
       [9.88236442e-01, 1.17635582e-02],
       [9.99940961e-01, 5.90387377e-05],
       [9.88867640e-01, 1.11323602e-02],
       [9.99940961e-01, 5.90393955e-05],
       [9.56047433e-01, 4.39525674e-02],
       [9.99940961e-01, 5.90393955e-05],
       [9.99940985e-01, 5.90154546e-05],
       [9.998931

In [9]:
lr.fit(x_train, y_train.ravel())
lr_pred = lr.predict_proba(x_test)[:,1]
lr_pred

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([8.75845531e-05, 4.80744469e-01, 2.04459742e-02, 4.46740526e-02,
       1.92961115e-02, 3.02787638e-03, 1.07830729e-03, 1.91191077e-03,
       7.66398078e-01, 7.56184092e-04, 2.54456794e-04, 4.75349749e-06,
       1.89430538e-02, 2.18834135e-10, 5.85561770e-05, 7.17740383e-04,
       9.96765499e-01, 1.43765075e-01, 1.44858996e-09, 1.63332335e-17,
       4.14154103e-07, 2.93420112e-04, 7.03011396e-02, 2.78871583e-03,
       4.49632620e-03, 5.83510553e-02, 1.28143545e-65, 9.66379078e-01,
       5.64225780e-01])

In [10]:
rf.fit(x_train, y_train.ravel())
rf_pred = rf.predict_proba(x_test)[:,1]
rf_pred

array([0.07      , 0.09733333, 0.        , 0.06066667, 0.2445    ,
       0.09066667, 0.425     , 0.        , 0.        , 0.03066667,
       0.        , 0.08      , 0.        , 0.33666667, 0.05      ,
       0.01      , 0.29666667, 0.332     , 0.        , 0.32      ,
       0.        , 0.19      , 0.        , 0.        , 0.08      ,
       0.08      , 0.255     , 0.04066667, 0.2545    ])

In [11]:
n_estimators = [50, 100, 150, 250, 300, 350]
max_depth = [1, 3, 5, 6, 7, 8, 9, 10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
grid_search1 = GridSearchCV(gdbt, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)
grid_result1 = grid_search1.fit(x_train, y_train.ravel())

grid_search2 = GridSearchCV(rf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)
grid_result2 = grid_search2.fit(x_train, y_train.ravel())

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:    6.3s finished
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:    7.9s finished


In [12]:
print("GDBT Best Accuracy: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
print("RF Best Accuracy: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))

GDBT Best Accuracy: 0.916667 using {'n_estimators': 50, 'max_depth': 1}
RF Best Accuracy: 0.916667 using {'n_estimators': 100, 'max_depth': 1}


In [13]:
gdbt_bestparam = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=grid_result1.best_params_['n_estimators'], max_features=19,
                                  max_depth=grid_result1.best_params_['max_depth'], learning_rate=0.03)
rf_bestparam = RandomForestClassifier(n_estimators=grid_result2.best_params_['n_estimators'], min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=grid_result2.best_params_['max_depth'], bootstrap=True)

In [14]:
gdbt_bestparam.fit(x_train, y_train.ravel())
gdbt_bestparam_pred = gdbt_bestparam.predict_proba(x_test)
gdbt_bestparam_pred

array([[0.92337293, 0.07662707],
       [0.92459676, 0.07540324],
       [0.94917719, 0.05082281],
       [0.92459676, 0.07540324],
       [0.89720707, 0.10279293],
       [0.91928349, 0.08071651],
       [0.59354801, 0.40645199],
       [0.94917719, 0.05082281],
       [0.94917719, 0.05082281],
       [0.92459676, 0.07540324],
       [0.94917719, 0.05082281],
       [0.94917719, 0.05082281],
       [0.94917719, 0.05082281],
       [0.75894581, 0.24105419],
       [0.94251147, 0.05748853],
       [0.9463849 , 0.0536151 ],
       [0.79415588, 0.20584412],
       [0.81296941, 0.18703059],
       [0.94917719, 0.05082281],
       [0.71123656, 0.28876344],
       [0.94917719, 0.05082281],
       [0.89195237, 0.10804763],
       [0.94917719, 0.05082281],
       [0.94917719, 0.05082281],
       [0.92843797, 0.07156203],
       [0.92843797, 0.07156203],
       [0.92233114, 0.07766886],
       [0.91928349, 0.08071651],
       [0.89720707, 0.10279293]])

In [15]:
rf_bestparam.fit(x_train, y_train.ravel())
rf_bestparam_pred = rf_bestparam.predict_proba(x_test)[:,1]
rf_bestparam_pred

array([0.07412737, 0.08137362, 0.04687796, 0.09138306, 0.11247463,
       0.10383551, 0.30450095, 0.04996129, 0.04595907, 0.10126028,
       0.04996129, 0.0609154 , 0.04595907, 0.36520554, 0.07722991,
       0.07516095, 0.16026081, 0.28299966, 0.04595907, 0.24836103,
       0.04595907, 0.07883086, 0.04595907, 0.04904241, 0.08137362,
       0.06265422, 0.12432658, 0.1006028 , 0.11464779])

In [16]:
def logdf(cols):
    df_test[cols] = df_test[cols].map(lambda i: np.log(i) if i > 0 else 0)

def dffillmed(cols):
    df_test[cols] = df_test[cols].fillna(df_test[cols].median())

ids = df_test['name']
df_test = df_test.drop(['email_address', 'name'] , axis=1)
df_test['deferred_income'] = df_test['deferred_income'].map(lambda i: np.log(-i) if -i > 0 else 0)

logdf('bonus')
logdf('deferral_payments')
logdf('director_fees')
logdf('exercised_stock_options')
logdf('expenses')
logdf('long_term_incentive')
logdf('other')
logdf('restricted_stock')
logdf('salary')
logdf('total_payments')
logdf('total_stock_value')

logdf('loan_advances')
logdf('restricted_stock_deferred')

dffillmed('from_poi_to_this_person')
dffillmed('from_messages')
dffillmed('to_messages')
dffillmed('shared_receipt_with_poi')
dffillmed('from_this_person_to_poi')

df_test.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,15.473738,14.57819,14.663280,0.0,13.767513,9.761636,484.0,228.0,108.0,0.0,0.000000,12.258181,11.967619,0.0,12.273727,5521.0,7991.0,15.520555,13.920506
1,14.115615,0.00000,6.725034,0.0,0.000000,11.096000,27.0,140.0,15.0,0.0,13.789467,7.390799,12.437403,0.0,12.537536,1593.0,1858.0,14.797435,12.437403
2,14.220976,0.00000,14.952385,0.0,15.527144,10.435262,32.0,32.0,21.0,0.0,14.296090,9.336973,13.656590,0.0,12.402022,1035.0,1045.0,12.573081,15.670411
3,14.914123,0.00000,0.000000,0.0,14.644548,11.364124,3069.0,66.0,609.0,0.0,14.074007,7.415175,14.095524,0.0,12.808099,2097.0,3093.0,15.373230,15.100398
4,13.815511,0.00000,12.367341,0.0,0.000000,10.331171,49.0,58.0,12.0,0.0,12.765688,12.637514,14.732626,0.0,12.936489,1585.0,1892.0,14.440785,14.732626


In [17]:
na_check(df_test)

,Missing Ratio


In [18]:
lr_predf = lr.predict_proba(df_test)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': lr_predf})
sub.to_csv('sub_lr.csv', index=False)
lr_predf

array([9.99999999e-01, 9.68947182e-01, 9.93296523e-01, 1.00000000e+00,
       9.72378567e-01, 5.05968027e-03, 9.99794615e-01, 9.86511638e-01,
       5.71784820e-01, 7.54244757e-01, 3.03530317e-41, 6.16963325e-03,
       9.26577623e-01, 3.72027363e-01, 1.10998753e-25, 1.62404107e-02,
       4.53405239e-02, 9.97434666e-01, 1.40380354e-01, 3.90449763e-08,
       2.87210878e-01, 9.99980602e-01, 7.64997369e-03, 5.59335744e-01,
       6.51621730e-01, 1.24926467e-03, 1.38590848e-07, 4.17809902e-04,
       5.90087818e-01, 9.55743193e-02, 5.07257340e-12, 1.04399472e-01,
       9.99970166e-01])

In [19]:
gdbt_predf = gdbt_bestparam.predict_proba(df_test)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': gdbt_predf})
sub.to_csv('sub_gdbt.csv', index=False)
gdbt_predf

array([0.30048996, 0.18075808, 0.24105419, 0.18075808, 0.1081153 ,
       0.24105419, 0.72213099, 0.06883414, 0.05748853, 0.09617848,
       0.06883414, 0.05082281, 0.05082281, 0.06883414, 0.31837859,
       0.05082281, 0.07254037, 0.101197  , 0.14327785, 0.06883414,
       0.86546764, 0.24105419, 0.06187537, 0.09617848, 0.07254037,
       0.05082281, 0.07156203, 0.07254037, 0.1081153 , 0.23088156,
       0.05082281, 0.101197  , 0.40445643])

In [20]:
rf_predf = rf_bestparam.predict_proba(df_test)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': rf_predf})
sub.to_csv('sub_rf.csv', index=False)
rf_predf

array([0.24927992, 0.22763443, 0.23215175, 0.23704052, 0.24553149,
       0.22197068, 0.56208721, 0.08647949, 0.09899494, 0.11257883,
       0.10263961, 0.04996129, 0.05806452, 0.08647949, 0.33435491,
       0.04904241, 0.11351692, 0.14177902, 0.11935128, 0.08739838,
       0.64687481, 0.25999875, 0.07910126, 0.11165994, 0.11259804,
       0.04904241, 0.08569507, 0.11351692, 0.13266311, 0.30565797,
       0.04996129, 0.1230613 , 0.26589256])

In [21]:
blending_pred = lr_predf*0.30  + gdbt_predf*0.67 + rf_predf*0.03
sub = pd.DataFrame({'name': ids, 'poi': blending_pred})
sub.to_csv('sub_blending_update.csv', index=False)
blending_pred

array([0.50880667, 0.4186211 , 0.46645981, 0.42821913, 0.37151676,
       0.16968333, 0.80062877, 0.34466675, 0.21302261, 0.29409037,
       0.04919807, 0.03740101, 0.31376651, 0.16032147, 0.2233443 ,
       0.04039468, 0.06560971, 0.37128576, 0.1416908 , 0.04874084,
       0.68543283, 0.46930045, 0.04612453, 0.2355901 , 0.24746651,
       0.03589734, 0.05051745, 0.0521329 , 0.25344349, 0.19253268,
       0.03555012, 0.10281367, 0.57895363])

In [22]:
from mlxtend.classifier import StackingClassifier

meta_estimator = GradientBoostingClassifier(tol=100, subsample=0.70, n_estimators=50, 
                                           max_features='sqrt', max_depth=4, learning_rate=0.3)
stacking = StackingClassifier(classifiers=[lr, gdbt_bestparam, rf_bestparam], 
                              meta_classifier=meta_estimator, use_probas=True, average_probas=False)

In [23]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

stacking.fit(x_train, y_train.ravel())
stacking_pred = stacking.predict_proba(df_test)[:,1]
sub = pd.DataFrame({'name': ids, 'poi': stacking_pred})
sub.to_csv('sub_stacking.csv', index=False)
stacking_pred

array([9.99896559e-01, 9.98785709e-01, 9.99905951e-01, 9.98785994e-01,
       9.99658427e-01, 1.48754558e-04, 9.99970611e-01, 9.77959467e-01,
       2.10748575e-04, 9.87672573e-01, 4.06338537e-05, 2.23316231e-05,
       3.39048583e-01, 3.03244699e-03, 2.69389094e-04, 2.23316231e-05,
       2.22621368e-05, 9.97761774e-01, 3.52982997e-03, 2.22554644e-05,
       9.99950144e-01, 9.99905973e-01, 2.23316412e-05, 9.89041591e-02,
       1.88117138e-01, 2.23316231e-05, 2.23236548e-05, 2.22615745e-05,
       9.89041591e-02, 9.63627776e-03, 2.23236366e-05, 7.51438912e-05,
       9.99971792e-01])